In [1]:
import os
import pandas as pd
import arcpy
from arcpy import env
from arcpy.management import *
from arcpy.sa import *
from arcpy.da import *
from arcpy.conversion import *
from arcpy.analysis import *

In [2]:
# tool function
# 创建渔网
def create_net(output_file,extent_file,cell_size):
    """参数说明：
        output_file: 输出文件
        extent_file: 输入范围文件
        cell_size: 生成的cell大小
"""
    proj = arcpy.Describe(extent_file).spatialReference
    extent_x_min = arcpy.Describe(extent_file).extent.XMin
    extent_y_min = arcpy.Describe(extent_file).extent.YMin
    extent_x_max = arcpy.Describe(extent_file).extent.XMax
    extent_y_max = arcpy.Describe(extent_file).extent.YMax
    with arcpy.EnvManager(outputCoordinateSystem = proj):
        arcpy.management.CreateFishnet(out_feature_class=output_file,origin_coord=f"{extent_x_min} {extent_y_min}",y_axis_coord=f"{extent_x_min} {extent_y_min+10}",cell_width=cell_size,cell_height=cell_size,number_rows=None,number_columns=None,corner_coord=f"{extent_x_max} {extent_y_max}",labels="NO_LABELS",template="#",geometry_type="POLYGON")
# 面转点
def polygon_point(in_feature, out_feature):
    """参数说明：
        in_feature: 输入面
        out_feature: 输出点
    """
    proj = arcpy.Describe(in_feature).spatialReference
    with arcpy.EnvManager(outputCoordinateSystem=proj):
        arcpy.management.FeatureToPoint(in_features=in_feature,out_feature_class=out_feature,point_location="INSIDE")

# 筛选点

def select_point(input_polygon,input_point,output_point):
    """参数说明：
        input_polygon: 选择面
        input_point: 待选择点
        output_point: 选择点
    """
    polygon_layer = "polygonLayer"
    point_layer = "pointLayer"
    arcpy.MakeFeatureLayer_management(input_polygon, polygon_layer)
    arcpy.MakeFeatureLayer_management(input_point, point_layer)
    arcpy.SelectLayerByLocation_management(in_layer=point_layer, overlap_type="INTERSECT", select_features=polygon_layer,search_distance=0,selection_type='NEW_SELECTION',invert_spatial_relationship="NOT_INVERT")
    # 导出选择的要素
    arcpy.CopyFeatures_management(point_layer, output_point)

# 添加字段
def add_field(input_table,field_name,field_type='TEXT'):
    """参数说明：
        input_table: 输入数据表
        field_name: 字段名
        field_type: 字段类型"""
    arcpy.AddField_management(input_table,field_name,field_type)

# 空间连接
def perform_spatial_join(target_layer_path, join_layer_path, output_layer_path, field_mapping_dict):
    """参数说明：
        target_layer_path: 目标图层路径
        join_layer_path: 连接图层路径
        output_layer_path: 输出图层路径
        field_mapping_dict: 字段映射字典
    """
    # 创建 FieldMappings 对象
    field_mappings = arcpy.FieldMappings()

    # 为每个字段对创建 FieldMap 并添加到 FieldMappings
    for target_field, join_field in field_mapping_dict.items():
        field_map = arcpy.FieldMap()
        
        # 添加目标字段和连接字段到FieldMap
        field_map.addInputField(target_layer_path, target_field)
        field_map.addInputField(join_layer_path, join_field)
        
        # 设置输出字段属性
        output_field = field_map.outputField
        output_field.name = target_field
        output_field.aliasName = target_field
        field_map.outputField = output_field

        # 将 FieldMap 添加到 FieldMappings
        field_mappings.addFieldMap(field_map)

    # 执行空间连接操作
    arcpy.analysis.SpatialJoin(target_layer_path, join_layer_path, output_layer_path,"JOIN_ONE_TO_ONE", "KEEP_ALL", field_mappings)


# 点采样
def sample_point(point_,raster_,out_name):
    """根据栅格采样点,输出为表格"""
    Sample(raster_,point_,out_name,"NEAREST", "OBJECTID", "CURRENT_SLICE", None, '', None, None, "ROW_WISE", "TABLE")
    return None

# 导出CSV
def export_csv(table_,out_path,out_name):
    """参数说明：
        table_: 输入数据表
        out_path: 输出路径
        out_name: 输出表名"""
    TableToTable(table_,out_path,out_name)
    return None


In [3]:
# 设置工作空间
env.workspace = r'D:\ArcGISProjects\workspace\shbyq\features_data_vector.gdb\features'


In [5]:
# 创建渔网
create_net('dunyun_grid','dy_tl',300)

In [6]:
# 要素转点
polygon_point('dunyun_grid','dunyun_inside_point')

In [7]:
# 按位置选择
select_point('dy_tl','dunyun_inside_point','select_point')

In [6]:
# 添加字段
input_features = 'select_point'
field_list = ['tl','yl','ts','tz']
for one_field in field_list:
    # 添加字段
    add_field(input_table=input_features,field_name=one_field,field_type='TEXT')

In [7]:
# 空间连接赋予地类属性
# 定义目标要素图层和连接要素图层的路径
targetFeatures = "select_point"
joinFeatures = "dy_tl"
outputFeatures = "join_point"
fields_mapping = {
    "TL": "土类",
    "YL": "亚类",
    "TS": "土属",
    "TZ": "土种"
}
# 空间连接
perform_spatial_join(targetFeatures,joinFeatures,outputFeatures,fields_mapping)

# 采样环节

In [18]:
# 对特征因子采样
# result table name
sample_csv_name = 'dy_frequency.csv'
# 选择用于采样的特征因子数据库
stander_raster_gdb = r"D:\ArcGISProjects\workspace\shbyq\features_data_dy.gdb"
env.workspace = stander_raster_gdb # 切换工作空间用于采样
# 用于采样的特征因子列表
feature_list = arcpy.ListRasters()
len(feature_list)

34

In [19]:
# 选择用于采样的要素类
point_data = r"D:\ArcGISProjects\workspace\shbyq\features_data_vector.gdb\features\join_point"

# 用于存放采样表的数据库路径
sample_gdb_path = r"D:\ArcGISProjects\workspace\frequency_filter"

In [30]:
# 采样点数据名称
sample_name =os.path.basename(point_data)
filed_list = [_.name for _ in arcpy.ListFields(point_data)]
print(filed_list)
# 需要保留的字段
elements_yes = ['OBJECTID', 'TL','YL','TS','TZ']
filter_list = [_ for _ in filed_list if _ in elements_yes]
print(filter_list)


['OBJECTID', 'Shape', 'Join_Count', 'TARGET_FID', 'TL', 'YL', 'TS', 'TZ']
['OBJECTID', 'TL', 'YL', 'TS', 'TZ']


In [22]:
# 使用Delete_management函数删除数据库中的所有内容
try:
    arcpy.Delete_management(os.path.join(sample_gdb_path,'tabledata.gdb'))
except:
    pass
# 再创建一个新的数据库
arcpy.management.CreateFileGDB(sample_gdb_path, "tabledata", "CURRENT")

<Result 'D:\\ArcGISProjects\\workspace\\frequency_filter\\tabledata.gdb'>

In [23]:
# 逐个采样并保存到csv文件
gdb_path = os.path.join(sample_gdb_path,"tabledata.gdb")
for one_feature in feature_list:
    print(one_feature)
    sample_point(point_data,one_feature,os.path.join(gdb_path,one_feature))

DEM
AnalyticalHillshading
Aspect
ChannelNetworkBaseLevel
ChannelNetworkDistance
ClosedDepressions
ConvergenceIndex
LSFactor
PlanCurvature
ProfileCurvature
RelativeSlopePosition
Slope
TopographicWetnessIndex
TotalCatchmentArea
ValleyDepth
Contrast
Correlation
Dissimilarity
Entropy
Homogeneity
Mean
ndvi
PCA_0
PCA_1
SecondMoment
Variance
PRE
SRA
TMP
VAP
WIND
BIO
LON
LAT


In [24]:
# 导出CSV表用于分析
env.workspace = gdb_path # 切换工作空间用于导出csv文件

In [31]:
# 读取数据表并保存到csv文件
result_df = pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(point_data,filter_list))
result_df.rename(columns={"OBJECTID":sample_name},inplace=True)

In [ ]:
#  读取每个表的最后一个字段的数据,存储每个表的最后一个字段的数据
for table in feature_list:
    # 将表转换为pandas数据帧
    df = pd.DataFrame(arcpy.da.TableToNumPyArray(table, "*"))  # 确保数据表中无空值
    # 提取最后一个字段的数据
    merged_df = df[[sample_name, df.columns[-1]]]
    # 合并
    result_df = pd.merge(result_df, merged_df, on=[sample_name])
# 保存到csv文件
result_df.rename(columns=dict(zip(result_df.columns[-len(feature_list):], feature_list)),inplace=True)
result_df.drop(result_df.columns[0],axis=1,inplace=True)
# result_df.rename(columns={"PH":"PH_T"},inplace=True)
result_df.to_csv(os.path.join(sample_gdb_path,sample_csv_name),index=False,encoding='utf-8')

In [34]:
df = pd.read_csv(r"D:\ArcGISProjects\workspace\frequency_filter\dy_frequency.csv")
df

,TL,YL,TS,TZ,DEM,AnalyticalHillshading,Aspect,ChannelNetworkBaseLevel,ChannelNetworkDistance,ClosedDepressions,...,SecondMoment,Variance,PRE,SRA,TMP,VAP,WIND,BIO,LON,LAT
0,黄壤,黄壤,硅铁质黄壤,薄腐殖层厚土层硅铁质黄壤,961.5000,0.706572,0.231111,827.27014,134.229860,NaN,...,0.506173,0.246914,101.916664,13154.333,15.766666,1.555833,1.658333,194.86105,443245.0,2861300.0
1,水稻土,潴育型水稻土,潴育型石灰性大眼泥田ARWC或APBC,大眼泥田,888.4000,0.426612,6.042653,870.42035,17.979675,NaN,...,0.234568,0.543210,101.916664,13154.333,15.766666,1.555833,1.658333,194.86105,443545.0,2861300.0
2,黄壤,黄壤,硅铁质黄壤,薄腐殖层厚土层硅铁质黄壤,870.8000,1.131889,1.264116,835.30994,35.490050,NaN,...,0.185185,0.913580,101.916664,13154.333,15.766666,1.555833,1.658333,194.86105,443845.0,2861300.0
3,黄壤,黄壤,硅铁质黄壤,薄腐殖层厚土层硅铁质黄壤,1021.6000,0.186124,5.645218,940.00836,81.591610,NaN,...,0.382716,0.444444,102.416664,13140.667,15.600000,1.540833,1.683333,195.54823,444445.0,2861300.0
4,黄壤,黄壤,硅铝质黄壤,薄腐殖层厚土层硅铝质黄壤,853.4000,0.811739,4.231506,814.98510,38.414917,NaN,...,0.160494,0.666667,101.833336,13150.667,15.750000,1.551667,1.675000,194.79027,442045.0,2861600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25293,黄壤,黄壤,硅铁质黄壤,薄腐殖层厚土层硅铁质黄壤,1185.2999,0.716325,0.344991,1137.37800,47.921875,NaN,...,0.506173,0.098765,98.833336,12810.917,14.716667,1.446667,1.758333,189.39268,448645.0,2924000.0
25294,黄壤,黄壤,硅铁质黄壤,薄腐殖层厚土层硅铁质黄壤,1104.7000,0.961624,1.333926,1085.43420,19.265747,NaN,...,0.358025,0.172840,98.833336,12810.917,14.716667,1.446667,1.758333,189.39268,448945.0,2924000.0
25295,黄壤,黄壤,硅铁质黄壤,薄腐殖层厚土层硅铁质黄壤,1070.0000,0.929952,3.877045,1041.28740,28.712646,NaN,...,0.432099,0.222222,98.833336,12810.917,14.716667,1.446667,1.758333,189.39268,449245.0,2924000.0
25296,黄壤,黄壤,硅铁质黄壤,薄腐殖层厚土层硅铁质黄壤,1104.7000,1.074531,2.214330,1055.28220,49.417725,NaN,...,1.000000,0.000000,98.500000,12832.167,14.908333,1.464167,1.750000,189.22100,450445.0,2924000.0
